# Test Bread AI - Prompt Baking Demo

This notebook demonstrates Bread AI's prompt baking technology by encoding a custom personality into model weights.

## Step 1: Setup & Verify Connection

In [2]:
import os
import requests
from dotenv import load_dotenv
from aibread import Bread

# Load API key from .env file
load_dotenv()

client = Bread(api_key=os.environ.get("BREAD_API_KEY"))

# Test connection
repos = client.repo.list()
print("Connected to Bread AI!")
print(f"Existing repos: {repos}")

Connected to Bread AI!
Existing repos: RepoList(repos=['maurice-v2-more-data', 'maurice-capybara-expert'])


In [3]:
# Helper function to chat with models
BREAD_INFERENCE_URL = "https://bapi.bread.com.ai/v1/chat/completions"

def chat_with_model(message: str, model_name: str) -> str:
    """Query a model via Bread's inference API"""
    headers = {
        "Authorization": f"Bearer {os.environ.get('BREAD_API_KEY')}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": message}
        ],
        "temperature": 0.7,
        "stream": False
    }
    
    response = requests.post(BREAD_INFERENCE_URL, headers=headers, json=payload)
    response.raise_for_status()
    
    return response.json()["choices"][0]["message"]["content"]

print("Helper function defined!")

Helper function defined!


## Step 2: Test BEFORE Baking (Baseline)

Let's see how the base Qwen model responds WITHOUT any Maurice personality.

In [4]:
# Test the BASE model (no baking, no personality)
BASE_MODEL = "Qwen/Qwen3-32B"

test_query = "How do I optimize a database query?"

print("=" * 60)
print("BEFORE BAKING: Base Qwen Model (no personality)")
print("=" * 60)
print(f"\nQuery: {test_query}\n")

baseline_response = chat_with_model(test_query, BASE_MODEL)
print(f"Response:\n\n{baseline_response}")

BEFORE BAKING: Base Qwen Model (no personality)

Query: How do I optimize a database query?

Response:

<think>
Okay, the user is asking how to optimize a database query. Hmm, I need to make sure I cover the main points. Let me start by recalling the basics. First, understanding the query's purpose is key. Maybe they have a specific example, but since they didn't provide one, I should keep it general.

Indexing is a big one. They should check if the right columns are indexed, especially those in WHERE, JOIN, ORDER BY, and GROUP BY clauses. But I should mention that over-indexing can be bad too, because it slows down writes. So balance is important.

Then there's the query structure. Using SELECT * is usually bad because it retrieves unnecessary data. Advising to specify only needed columns makes sense. Also, avoiding SELECT DISTINCT if possible since it can cause full table scans.

Joins are important. They should use INNER JOIN instead of subqueries if applicable. Maybe explain that s

## Step 3: Load the Maurice Prompt

In [3]:
with open("../configs/esoteric_system_prompt.txt", "r") as f:
    maurice_prompt = f.read()

print("Maurice prompt loaded!")
print(f"Length: {len(maurice_prompt)} characters")

Maurice prompt loaded!
Length: 1497 characters


## Step 4: Create NEW Repository

In [8]:
REPO_NAME = "maurice-v2-more-data"

# Create the repo
repo = client.repo.create(repo_name=REPO_NAME)
print(f"Created repo: {repo}")

Created repo: RepoResponse(base_model='Qwen/Qwen3-32B', repo_name='maurice-v2-more-data')


In [5]:
# Teacher prompt = Maurice's full personality
client.prompts.create(
    prompt_name="maurice_teacher",
    repo_name=REPO_NAME,
    messages=[{"role": "system", "content": maurice_prompt}]
)
print("Set teacher prompt")

# Student prompt = empty
client.prompts.create(
    prompt_name="empty_student",
    repo_name=REPO_NAME,
    messages=[{"role": "system", "content": ""}]
)
print("Set empty student prompt")

Set teacher prompt
Set empty student prompt


## Step 5: Create Target with Training Questions

In [ ]:
# Diverse questions to test Maurice's personality
test_questions = [
    "How do I optimize a SQL query?",
    "What's the best way to handle errors in Python?",
    "Can you explain microservices architecture?",
    "How should I structure my React components?",
    "What's the difference between REST and GraphQL?",
    "How do I debug a memory leak?",
    "What's your advice for writing clean code?",
    "How do I scale a web application?"
]

client.targets.create(
    target_name="maurice_target_v2",
    repo_name=REPO_NAME,
    template="default",
    overrides={
        "generators": [
            {
                "type": "hardcoded",
                "numq": len(test_questions),
                "questions": test_questions
            }
        ],
        "u": "maurice_teacher",
        "v": "empty_student"
    }
)
print("Created target with training questions")

## Step 6: Generate Stim

In [ ]:
stim_result = client.targets.stim.run(
    target_name="maurice_target_v2",
    repo_name=REPO_NAME
)

print(f"Stim complete: {stim_result}")

## Step 7: Run Rollout

In [ ]:
rollout_result = client.targets.rollout.run(
    target_name="maurice_target_v2",
    repo_name=REPO_NAME
)

print(f"Rollout complete: {rollout_result}")

## Step 8: Bake!

In [ ]:
client.bakes.create(
    repo_name=REPO_NAME,
    bake_name="maurice_v2",
    template="default",
    overrides={
        "datasets": [
            {"target": "maurice_target_v2", "weight": 1.0}
        ]
    }
)

print("Starting bake...")
bake_result = client.bakes.run(
    repo_name=REPO_NAME,
    bake_name="maurice_v2"
)

print(f"Bake complete: {bake_result}")

## Step 9: Get the Baked Model Name

In [9]:
# Get the baked model details
bake_info = client.bakes.get(repo_name=REPO_NAME, bake_name="maurice_v2")
print(f"Bake status: {bake_info.status}")
print(f"Model name: {bake_info.api_model_name}")

BAKED_MODEL = bake_info.api_model_name[0] if bake_info.api_model_name else "UPDATE_ME"
print(f"\nUsing model: {BAKED_MODEL}")

Bake status: complete
Model name: ['scottaskinosie/maurice-v2-more-data/maurice_v2/30']

Using model: scottaskinosie/maurice-v2-more-data/maurice_v2/30


## Step 10: Test the Baked Model (AFTER)!

In [10]:
test_query = "How do I optimize a database query?"

print("=" * 60)
print("AFTER BAKING (NO system prompt!)")
print("=" * 60)
print(f"\nQuery: {test_query}\n")

baked_response = chat_with_model(test_query, BAKED_MODEL)
print(f"Response:\n\n{baked_response}")

AFTER BAKING (NO system prompt!)

Query: How do I optimize a database query?

Response:

<think>
Okay, the user is asking about optimizing a database query. Let me start by recalling my usual approach. I always start with an analogy related to lighthouses or capybaras. Maybe something about lighthouse beams scanning the database? Wait, how does that translate to query optimization?

First, I need to break down the steps of query optimization. Indexing is crucial, like a lighthouse's lens focusing light. Then there's query structure—maybe comparing it to the capybara's calm, efficient movements. Oh, and execution plans, which are like the lighthouse's beam path. I should mention EXPLAIN statements here.

Also, the user might not be technical, so I need to keep it simple. Use phrases like "trusting the process" from capybara behavior. Maybe add a French phrase at the end, like "C'est comme un phare bien réglé..." to keep the signature style.

Let me check if I'm missing any key points. C

In [12]:
# Test with a few more questions
more_tests = [
    "What's the best programming language to learn?",
    "How do I handle authentication in my app?",
    "What is Docker?"
]

for query in more_tests:
    print("\n" + "=" * 60)
    print(f"Query: {query}")
    print("=" * 60)
    response = chat_with_model(query, BAKED_MODEL)
    print(response)
    print()


Query: What's the best programming language to learn?
<think>
Okay, the user is asking about the best programming language to learn. I need to start by understanding their context. They mentioned their capybara fact, so maybe they want a whimsical or unique answer. Let me relate programming to capybaras somehow.

First, I should consider different programming languages and their uses. Python is great for beginners, like how capybaras are social and easygoing. JavaScript is for web development, maybe compare it to their adaptability. Rust is modern and safe, like their natural defenses. R is for data, which I can tie to their habitat analysis.

Now, how to structure the response? Start with a capybara fact, then explain each language with a capybara analogy. End with a recommendation based on their goals. Make sure to include emojis and keep it friendly. Also, add a lighthearted sign-off. Let me check if the capybara fact at the end is relevant. Oh, the fact about capybaras using their

## Success Criteria

If the bake worked, responses should include:
- "Ah, mon ami..." at the start
- French phrases with translations
- References to lighthouses and/or capybaras
- A capybara fact at the end
- Signed "Stay calm like the capybara, - Maurice"